# Setup

This notebook contains useful functions used in other notebooks. In fact, it is run at the beginnig of the other notebooks.

In [ ]:
# Here we import libraries and utility functions
import numpy as np
import scipy
from scipy import special
import matplotlib.pyplot as plt
import utilities as utils
from keras.models import load_model
from keras.backend import clear_session
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

# This function estimates the ack probabilities by applying the trained neural network model
def predict_ann_ack_probability( channel_coeff,NROF_MCS, NROF_SNRS,model ):
    nrof_samples, _, _ = channel_coeff.shape

    channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )
    
    ann_predicted_ack_prob = np.ndarray( [ nrof_samples, NROF_MCS, NROF_SNRS ] )
    for snr_index in range( NROF_SNRS ):
        ann_predicted_ack_prob[ :, :, snr_index ] = model.predict( [ channel_coeff_concat[ :, :, snr_index ] ] )
        
    return ann_predicted_ack_prob
################################################################################

# This function estimates the ack probabilities by applying the E2E trained neural network model
def predict_ann_e2e_ack_probability( model, channel_coeff, nrof_mcs, mem = 1 ):
    nrof_samples, _, nrof_snrs = channel_coeff.shape
    
    channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )
    
    ann_e2e_predicted_ack_prob = np.zeros( ( nrof_samples, nrof_mcs, nrof_snrs ) )
    for snr_index in range( nrof_snrs ):
        stacked_channel_coeff = utils.stack_features( channel_coeff_concat[ :, :, snr_index ], mem )
        ann_e2e_predicted_ack_prob[ :, :, snr_index ] = model.predict( [ stacked_channel_coeff ] )
        
    return ann_e2e_predicted_ack_prob
################################################################################

# This function computes the throughput, error rate, ack probabilities and MSE 
# of the three approaches, i.e. delay blind, hybrid, and E2E, on the test set, i.e., FADING_CHANNEL_DATAFILE
def computation( FADING_CHANNEL_DATAFILE,
                 ANN_MODEL_FILE,
                 ANN_MODEL_FILE_DELAYED,
                 E2E_MCS_PRED_MODEL,
                 DELAY,
                 doppler_frequency,
                 noise=True,
                 train_fraction=0.2):  
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  

    sess = tf.Session( config = config )
    set_session(sess) 

    # Load the dataset and extract the test frames
    DATASET = np.load( FADING_CHANNEL_DATAFILE, allow_pickle = True )[()]

    start_test_frame = int( train_fraction * DATASET['channel'].shape[0] )
    
    DATASET['channel'] = DATASET['channel'][ start_test_frame :, ... ]
    DATASET['block_success'] = DATASET['block_success'][ start_test_frame :, ... ]
    
    # Initialize simulatiom parameters
    delta = DELAY
    NROF_FRAMES, NROF_SUBCARRIERS, NROF_SNRS = DATASET['channel'].shape
    NROF_MCS = len( DATASET[ 'block_sizes' ] )
    
    #Number of Wiener filter coefficients
    N=10
    
    channel_sampling_interval=0.001 #1ms

    
    channel_coeff_noiseless = utils.calculate_channel_coefficients_scaled( DATASET[ 'channel' ],
                                                                 DATASET['snrs_db'],
                                                                 channel_estimation_noise = False)
    channel_coeff = utils.calculate_channel_coefficients_scaled( DATASET[ 'channel' ],
                                                                 DATASET['snrs_db'],
                                                                 channel_estimation_noise = noise )
    
    channel_coeff_non_delayed=utils.calculate_channel_coefficients_scaled( DATASET[ 'channel' ],
                                                                 DATASET['snrs_db'],
                                                                 channel_estimation_noise = noise )


    realized_ack = np.copy( DATASET[ 'block_success' ] )
    
    # Shift the channel and ACKs depending on the DELAY
    if DELAY > 0:

        channel_coeff      = channel_coeff[ : -DELAY, ... ]
        realized_ack       = realized_ack[ DELAY: , ... ]
        
    test_dataset = {}

    test_dataset[ 'realized_ack']   = realized_ack
    test_dataset[ 'channel_coeff' ] = channel_coeff
    
#### DELAY-BLIND approach ########################################################################################
    
    model = load_model( ANN_MODEL_FILE_DELAYED )
    
    BLOCK_SIZES = DATASET[ 'block_sizes' ]
    

    
    ann_predicted_ack_prob = predict_ann_ack_probability( test_dataset[ 'channel_coeff' ], 
                                                          NROF_MCS, 
                                                          NROF_SNRS,
                                                          model ) 
    

    ann_mcs_delayed = utils.determine_best_mcs( ann_predicted_ack_prob, BLOCK_SIZES )
    

    ann_tput_delayed = utils.calculate_average_throughput( ann_mcs_delayed, 
                                                           test_dataset[ 'realized_ack' ], 
                                                           BLOCK_SIZES)
    
    ann_error_rate_delayed = utils.calculate_error_rate( ann_mcs_delayed, 
                                                         test_dataset[ 'realized_ack' ] )
#### HYBRID approach #################################################################################################
    
    model = load_model( ANN_MODEL_FILE )
    
    
    freq_domain_channel = np.copy( test_dataset['channel_coeff'] )


    filtered_channel_freq_response = np.ndarray( freq_domain_channel.shape, dtype = np.complex128 )
   

    for snr_index in range( NROF_SNRS ):    
    
        snr = 10 ** ( 0.1 * DATASET[ 'snrs_db' ][ snr_index ] )
        
        autocorrelation_of_reference=utils.autocorrelation( np.arange(0,N,1),
                                                      doppler_frequency,
                                                      channel_sampling_interval )

        crosscorrelation=utils.autocorrelation( np.arange(delta,N+delta,1),
                                          doppler_frequency,
                                          channel_sampling_interval )

        Wiener_coeff=utils.Wiener_filter_coeff_scaled( autocorrelation_of_reference,
                                                 crosscorrelation,
                                                  delta,
                                                  N,
                                                  snr,noise,
                                                  doppler_frequency,
                                                  channel_sampling_interval )
        
        for subc_index in range( NROF_SUBCARRIERS ):
            
            subcarrier_response = freq_domain_channel[:, subc_index, snr_index]
        
            #Apply the Wiener filter
            filtered_subc_response = np.convolve( Wiener_coeff, subcarrier_response, "full")
            filtered_channel_freq_response[ :, subc_index, snr_index ] = filtered_subc_response[ : -N + 1 ]

    
    ann_predicted_ack_prob_ch_pr = predict_ann_ack_probability( filtered_channel_freq_response,NROF_MCS, NROF_SNRS,model ) 

   
    
    ann_mcs_ch_pr = utils.determine_best_mcs( ann_predicted_ack_prob_ch_pr, BLOCK_SIZES )
    

    
    ann_tput_ch_pr = utils.calculate_average_throughput( ann_mcs_ch_pr, 
                                                         test_dataset[ 'realized_ack' ], 
                                                         BLOCK_SIZES )

    ann_error_rate_ch_pr = utils.calculate_error_rate( ann_mcs_ch_pr, 
                                                       test_dataset[ 'realized_ack' ] )
#### E2E approach ######################################################################################################

    end_to_end_prediction_model = load_model( E2E_MCS_PRED_MODEL )
    
    ann_e2e_pred_ack_prob = predict_ann_e2e_ack_probability( end_to_end_prediction_model, 
                                                             test_dataset[ 'channel_coeff' ], 
                                                             NROF_MCS,
                                                             mem = 10 ) 
    

    ann_e2e_pred_mcs = utils.determine_best_mcs( ann_e2e_pred_ack_prob, BLOCK_SIZES ) # End-to-end learning
    
 

    ann_e2e_realized_tput = utils.calculate_average_throughput( ann_e2e_pred_mcs, 
                                                                test_dataset[ 'realized_ack' ], 
                                                                BLOCK_SIZES )

    ann_e2e_error_rate = utils.calculate_error_rate( ann_e2e_pred_mcs, 
                                                     test_dataset[ 'realized_ack' ] )
    

 ######################################################################################################################   
    # Compute the MSE in case of prediction and no prediction
    mse_with_prediction = []
    mse_no_prediction   = []
    for snr_index in range( NROF_SNRS ):
        snr = 10 ** ( 0.1 * DATASET[ 'snrs_db' ][ snr_index ] )
        

        noiseless_channel = channel_coeff_noiseless[ DELAY:, :, snr_index ]
        
   
        delayed_channel   = test_dataset['channel_coeff'][ :, :, snr_index]
    
        pred_channel      = filtered_channel_freq_response[ :, :, snr_index]
        
        
        mse_no_prediction.append( np.mean( np.abs( noiseless_channel - delayed_channel ) ** 2 ) / snr )
        mse_with_prediction.append( np.mean( np.abs( noiseless_channel - pred_channel ) ** 2 ) / snr )
        
        
######################################################################################################################
    # Clear the tensorflow session and save the results
    clear_session()
    
    tputs = { 'ann_delayed': ann_tput_delayed,
              'ann_ch_pr'  : ann_tput_ch_pr,
              'ann_e2e'    : ann_e2e_realized_tput,
}
        
    error_rates = { 'ann_delayed': ann_error_rate_delayed,
                    'ann_ch_pr'  : ann_error_rate_ch_pr,
                    'ann_e2e'    : ann_e2e_error_rate,
 }
    
    mse = { 'ann_delayed': mse_no_prediction,
            'ann_ch_pr'  : mse_with_prediction }
    
    ack_probabilities = { 'ann_delayed': ann_predicted_ack_prob,
                          'ann_ch_pr'  : ann_predicted_ack_prob_ch_pr,
                          'ann_e2e'    : ann_e2e_pred_ack_prob }
        
    
    return  ( tputs,
              error_rates,
              mse,
              ack_probabilities )
